In [12]:
using LinearAlgebra
using FromFile
using FrankWolfe
include("auxiliary_functions.jl");

In [11]:
a = ones(Float64, 2, 3)

2×3 Matrix{Float64}:
 0.0  0.0  0.0
 0.0  0.0  0.0

In [13]:
a = vcat(a, zeros(Float64, 1, size(a, 2)))

3×3 Matrix{Float64}:
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0

In [16]:
a * ones(Float64, 3, 1)

3×1 Matrix{Float64}:
 0.0
 0.0
 0.0

In [138]:
function L2Loss(
    data,
    labels;
    lmbda=0.0,
    data_squared=nothing,
    labels_squared=nothing,
    data_squared_inverse=nothing,
    data_labels=nothing)

    A = data
    m, n = size(A)
    b = reshape(labels, m, 1)  # reshape into column vector
    lmbda = Float64(lmbda)

    # A squared
    if data_squared != nothing
        A_squared = (2 / m) * data_squared
    else
        A_squared = (2 / m) * (A' * A)
    end

    if lmbda != 0.0
        A_squared = A_squared + lmbda * Matrix(I, n, n)
    end

    @assert size(A_squared) == (n, n)

    # A.Tb
    if data_labels != nothing
        A_b = (2 / m) * data_labels
        A_b = reshape(A_b, n, 1)
    else
        A_b = (2 / m) * (A' * b)
        A_b = reshape(A_b, n, 1)
    end
    
    
    # A_squared_inv
    A_squared_inv = nothing
    solution = nothing
    if data_squared_inverse != nothing
        A_squared_inverse = (m / 2) * data_squared_inverse
        solution = data_squared_inverse * data_labels
        @assert lmbda == 0. "Regularization not implemented for hessian-based algorithms."
    end

    # b.Tb
    if labels_squared != nothing
        b_squared = (2 / m) * b_squared
    else
        b_squared = (2 / m) * (b' * b)
    end
    
    """
    evaluates f at x
    """
    function evaluate_function(x::Vector{Float64})
        # make x column vector
        x = hcat(x)

        # return value instead of 1x1 Matrix
        return ((1 / 2) * (x' * A_squared * x) + (A_b' * x) + (1 / 2) * b_squared)[1]
    end

    """
    evaluates gradient of f at x
    """
    function evaluate_gradient!(storage::Vector{Float64}, x::Vector{Float64})
        # make x column vector
        x = hcat(x)
        return storage .= A_squared * x + A_b
    end

    return evaluate_function, evaluate_gradient!
    
end
    

L2Loss

In [122]:
A = Matrix(I, 3, 3)
b = ones(3, 1)

3×1 Matrix{Float64}:
 1.0
 1.0
 1.0

In [139]:
f, grad! = L2Loss(A, b)


(var"#evaluate_function#67"(Core.Box([2.0;;]), Core.Box([0.6666666666666666; 0.6666666666666666; 0.6666666666666666;;]), Core.Box([0.6666666666666666 0.0 0.0; 0.0 0.6666666666666666 0.0; 0.0 0.0 0.6666666666666666])), var"#evaluate_gradient!#68"(Core.Box([0.6666666666666666; 0.6666666666666666; 0.6666666666666666;;]), Core.Box([0.6666666666666666 0.0 0.0; 0.0 0.6666666666666666 0.0; 0.0 0.0 0.6666666666666666])))

In [124]:
a = Matrix(I, 3, 1)
b = ones(Int64, 3, 1)
a' * b

1×1 Matrix{Int64}:
 1

In [125]:
hcat(hcat([1, 2, 3]))

3×1 Matrix{Int64}:
 1
 2
 3

In [126]:
grad!(Vector(x[:, 1]), x)

3-element Vector{Float64}:
 1.3333333333333333
 2.0
 2.6666666666666665

In [146]:
lmo = FrankWolfe.ProbabilitySimplexOracle(1.)
x0 = [1., 0, 0]

3-element Vector{Float64}:
 1.0
 0.0
 0.0

In [147]:
frank_wolfe(f, grad!, lmo, x0)

([0.33333328648682414, 0.333333304241379, 0.33333340927179683], [1.0, 0.0, 0.0], 1.7777777777777808, 3.123101188773347e-8, Any[])

In [2]:
using LinearAlgebra
a = 1 * Matrix(I, 3, 1)

3×1 Matrix{Int64}:
 1
 0
 0

In [3]:
b = 2*copy(a)

3×1 Matrix{Int64}:
 2
 0
 0

In [4]:
a + b

3×1 Matrix{Int64}:
 3
 0
 0

In [5]:
a .+ b

3×1 Matrix{Int64}:
 3
 0
 0